## Chapter 2-5, 2강 TTS(텍스트→음성) 기초와 보이스 클로닝 — Coqui TTS

- 목표: TTS 기본 파이프라인 이해, 멀티스피커/멀티언어 모델 사용, 간단 보이스 클로닝 실습
- 데이터: 짧은 텍스트 문장과 참조 화자 오디오(30~60초)
- 규칙(강의용): 시각화는 `matplotlib`만 사용, 불필요한 외부 의존 최소화


#### 구성
- TTS 개요 및 파이프라인
- 환경 준비(라이브러리)와 폰트/경고 설정
- 데이터 준비: 참조 화자 오디오 전처리(16kHz/모노)
- Coqui `xtts_v2`/`YourTTS` 데모: 기본 합성, 보이스 클로닝
- 간단 품질 점검 포인트(발음/억양/잡음)



### 0. 환경 준비 및 라이브러리 임포트

- 시각화는 `matplotlib`만 사용합니다.
- TTS는 `TTS`(Coqui), 오디오 I/O/변환은 `torchaudio`를 사용합니다.
- 한글 폰트와 경고 억제를 설정합니다.



In [ ]:
# 선택: 필요한 패키지 설치 (이미 설치되어 있으면 건너뜀)
import importlib, sys, subprocess

def ensure(pkg: str, pip_name: str | None = None):
    name = pkg if pip_name is None else pip_name
    try:
        importlib.import_module(pkg)
    except Exception:
        print(f'Installing {name} ...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', name, '--quiet'])

# torchaudio는 플랫폼별 제약이 있어 사전 설치 권장
ensure('TTS')  # Coqui TTS
try:
    import torchaudio
except Exception:
    print('torchaudio가 필요합니다. 환경에 맞게 설치하세요.')



In [ ]:
# -*- coding: utf-8 -*-
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchaudio

try:
    from TTS.api import TTS as COQUI_TTS
    _HAS_TTS = True
except Exception:
    _HAS_TTS = False

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=r"Glyph.*missing from font.*", category=UserWarning)

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', DEVICE)



### 1. 데이터 준비 및 전처리
- 참조 화자 오디오 로딩, 16kHz/모노 정규화
- 텍스트 예문 준비



In [ ]:
def ensure_mono_16k(path: str) -> str:
    """
    파일을 로드해 16kHz/모노로 저장한 임시 경로를 반환합니다.
    """
    if not os.path.exists(path):
        raise FileNotFoundError('참조 화자 오디오를 찾을 수 없습니다: ' + path)
    wav, sr = torchaudio.load(path)
    if wav.size(0) > 1:
        wav = wav.mean(dim=0, keepdim=True)
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        wav = resampler(wav)
        sr = 16000
    out_path = 'ref_16k_mono.wav'
    torchaudio.save(out_path, wav, sr)
    return out_path

REF_WAV = 'speaker_ref.wav'  # 수강생 교체 지시
SAMPLE_TEXT = '안녕하세요. 오늘은 텍스트를 음성으로 합성해 보겠습니다.'



### 2. Coqui TTS 데모 — 기본 합성과 보이스 클로닝
- `xtts_v2`로 보이스 클로닝(TTS API)
- 참조 화자 오디오 1개로도 가능하며, 2~3개면 안정성↑



In [ ]:
def synthesize_tts_xtts(text: str, speaker_wavs: list[str], out_path: str = 'tts_out.wav', language: str = 'ko'):
    if not _HAS_TTS:
        raise ImportError('Coqui TTS가 설치되어 있지 않습니다. 설치: pip install TTS')
    tts = COQUI_TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=(DEVICE=='cuda'))
    tts.tts_to_file(text=text, speaker_wav=speaker_wavs, language=language, file_path=out_path)
    return out_path

# 데모 (주석 해제 후 사용)
# 권장: 매우 짧은 문장으로 먼저 테스트(지연 시간 단축)
# try:
#     ref = ensure_mono_16k(REF_WAV)
#     out_wav = synthesize_tts_xtts('안녕하세요.', [ref], out_path='tts_demo.wav', language='ko')
#     print('합성 완료:', out_wav)
# except Exception as e:
#     print('데모 실행 중 문제:', e)



### 3. 품질 점검 포인트
- 발음 오류: 숫자/기호/영어 혼용 시 텍스트 정규화
- 억양/간투어: 문장 분할 및 문장부호 점검
- 잡음/메탈릭: 참조 음성의 녹음 품질(무향실/마이크) 영향



### 4. 마무리 및 과제 제안
- 참조 화자 오디오 1~2개로 클로닝 품질 비교
- 문장 길이/문장부호/언어를 바꾸며 합성 결과 청취 비교

